# Introduction to `SEDBuilder`

### Welcome! This notebook hopefully provides a sufficient and simple introduction to what `SEDBuilder` does and how to use it. Enjoy, and let me know if you have any questions - Zephyr

### Important: `SEDBuilder` currently only works for objects in the 2MASS survey (includes almost all stars), because it uses the 2MASS ID as a unique object identifier. 

In [ ]:
# You don't have to use autoreload, it's just useful for me while I'm still developing the package.
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function #ONLY NECESSARY IN PYTHON 2
import SEDBuilder.spectra  # Very occasionally astroquery.gaia (loaded by SEDBuilder) doesn't import correctly. 
                            # If so just re-run this.
                            # As well as SEDBuilder you'll need to have numpy, astropy, and astroquery installed 
                            # (and possibly at some point matplotlib).
import numpy as np
import astropy
import os

# optional, for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Examples of using `SEDBuilder`

### Let's use v410 x-ray 1 as an example. 
### We'll idenitfy it by one (of many) of the names Simbad can use to recognise it.


In [ ]:
v410xray1 = '[SS94] V410 X-ray 1' # (could also use the coordinates, ra+dec, or any other Simbad-compatible name)

# getInfo()
### Pulls simple data about this star from Simbad (+Gaia) and puts it into a Python dictionary.

In [ ]:
starInfo=SEDBuilder.spectra.getInfo(name=v410xray1)
# If you'd rather use a coordinate call, getInfo(ra=RA,dec=DEC) will return data for nearest 2MASS object to the specified RA and dec.

# And let's have a look at that data (returned as a dictionary)
print('\n _______INFORMATION ABOUT THE STAR')
for key in starInfo.keys():
    print(key,': ',starInfo[key])
# For this particular star, Gaia has no recorded radius, luminosity or dust-reddening coefficient (Ag). 
# (However, they have calculated that info for many stars, so worth it is still worth checking.)

# getSED()
### Finds the photometric data points for this object that are tabulated in VizieR and NASA's IRSA database, as well as supplementary metadata of the type returned by getInfo().
### It returns the photometric data points in an astropy table, with the supplementary metadata in that table's table.meta attribute.
### It should take a few seconds to run.


In [ ]:
table=SEDBuilder.spectra.getSED(name=v410xray1)

# Again let's have a look at that data
print('\n _______MAIN TABLE OF FLUXS:')
print(table)
print('\n _______SUPPLEMENTARY INFORMATION') # almost exactly the same as what is returned by getInfo()
for key in table.meta.keys():
    print(key,': ',table.meta[key])

### We try our best to identify which telescope each observation is from, but sometimes it's not clearly recorded (or it's just a one-off observation and not from a recognizable survey). You can add new telescopes, or link certain papers with telescopes, by hardcoding them in the getSED function, or asking me to do so.

### For a lot of users, this may be all you need (if you just want an object's SED), but below there's details about how you can do even more by saving the photometric data points to a file.

# Saving to file and adding new data

### Great. Now we could just use the data like this, in an astropy table. However if we save it to file we can do even more, as we can start appending data from other sources (right now we're limited to just those that are easily sourceable and machine-readable online).

### First let's just save it to file:

In [ ]:
# Finds data repository in the install location of the SEDBuilder package.
# (though of course you can specify any repository as saveDirectory instead.)
SEDDir = os.path.abspath(SEDBuilder.__file__)
SEDDir = "/".join(SEDDir.split("/")[:-2])
saveDirectory=SEDDir+'/spectraData'

# Writes data to a file (identified by the star's 2MASS Id) in that directory.
# THIS IS CURRENTLY SET TO OVERWRITE EXISTING DATA!
# Overwrite behavior:
# overwrite = -1 : overwrites nothing
# overwrite = 0  [default]: overwrites data only if there's a more recent data point available online.
# overwrite = 1  : overwrites everything
table = SEDBuilder.spectra.getSED(name=v410xray1,saveDir=saveDirectory,overwrite=1)

# Let's have a look at that data.
print('\n _______MAIN TABLE OF FLUXS:')
print(table)
print('\n _______SUPPLEMENTARY INFORMATION')
for key in table.meta.keys():
    print(key,': ',table.meta[key])

### Notice, the SED data is the same, but there's a bit more supplementary data (mostly blank fields we may want to fill in later).

### Great, now let's add some data about this star from other sources. There are three functions we can use here:
* addToMetaData(): add or replace a field in the star's metadata.
* addRegionToMetaData(): add's a star's "region" to its metadata. This is separate because a star can belong to many regions, while no other metadata field should ever have more than one value.
* addToSED(): add data point(s) to the star's SED. 

In [ ]:
# Already there's some useful info stored in the file and the table's metadata, but let's add more.

# There are two types of metadata we can add:
# 1. the object's on-sky region, which may take on multiple values (e.g. an object may belong to both "Eta-Cha" and "Cha-I")
# 2. any other metadata, which should have a single unique value.

# First, the on-sky region:
twoMassId = table.meta['2MASSID'] # This is the universal identifier used for the star.
Source='Andrews+ 2013'
Region='Taurus'
SEDBuilder.spectra.addRegionToMetadata(twoMassId,saveDirectory,Region,Source) # Special version of addToMetadata
# If you want to keep track of which papers refer to a specific star, you can keep calling addRegionToMetadata,
# and it will just add more sources. Again, all other metadata fields can only be specified once, with one source.


# Next, some other data about the star from the above source.
Luminosity=0.45 # in Lsun
Teff=3270 # in Kelvin
Av=3.67

# We could add these fields individually, e.g.:
SEDBuilder.spectra.addToMetadata(twoMassId,saveDirectory,'Luminosity',Luminosity,Source,overwrite=0)

# Or we can save some effort by adding them as a list:
SEDBuilder.spectra.addToMetadata(twoMassId,saveDirectory,['Teff','Av'],[Teff,Av],Source,overwrite=0)
# Note - with overwrite=0 [default], this function call won't actually add the new Teff to the file,
# because this star has a newer Teff value from Gaia. Instead it will return a warning and continue.
# If you really wanted to add it, you could set overwrite=1 in the function call.
# (This behaviour occurs a lot in SEDBuilder--I want to make it hard - but possible - for the user to overwrite 
# good data, and close to impossible to duplicate data.)


# What about some metadata from a whole bunch of sources? Easy.
sources=['Siess+ 2000','Siess+ 2000','Boneberg+ 2018']
fields=['Mass','Age','DiskRadius']
values=[0.245,1.29,0.6] # in Msun, Gyr and AU respectively
SEDBuilder.spectra.addToMetadata(twoMassId,saveDirectory,fields,values,sources)

# Finally, let's add a data point to the SED (in the form of a numpy array. Here, we only have one data point, but this function could add many at a time.)
l=np.array([1.3e-3]) # wavelength in m
f=np.array([0.3*1e-29*3e8/(l**2)]) # flux in W m (here converting from a value given for F_nu in mJ)
e=np.array([0]) # error in flux in W m. An error of 0 means the point is an upper limit.
SEDBuilder.spectra.addToSED(twoMassId,saveDirectory,l,f,e,'Boneberg+ 2018','ALMA')

# Retrieve all this data as an astropy table (actually, all the above functions return this, but it's conceptually easier to just retrieve it at the end.)
table=SEDBuilder.spectra.getSEDFromFile(twoMassId,saveDirectory)

# One last look at the data
print('\n _______MAIN TABLE OF FLUXS:')
print(table)
print('\n _______SUPPLEMENTARY INFORMATION')
for property in table.meta:
    print(property,': ',table.meta[property])

In [ ]:
#How many references have we just collated to get the SED data about this object?
print(np.unique(table["source"]))

### Finally, try running this cell again. Notice that it warns you that you're trying to duplicate data.

### It won't break when you do this, it just ignores your request and continues. (In most places you can set overwrite=1 to force it to overwrite/duplicate). 

### Overwriting is a bit scary generally, if you're going to do it I suggest starting from scratch (running getSED() with overwrite=1 and appending brand new data).

In [ ]:
# Now, let's plot the results: the SED for this object.

In [ ]:
exampleFig=plt.figure(figsize=(8,6))
examplePlot=plt.gca()
SEDBuilder.spectra.plotSED(examplePlot,table)
plt.show()

# Notes and extras:

- This tool is, by design, very agnostic. It collects all the data it can find. The data is not cleaned, it comes from many sources and some may not be using the same conventions. The philosphy is that with enough data these variations become invisible. This is a bad philosphy. The point of SEDBuilder is to make SED data accesesible, visible and usable, but be very wary that the flood of data may be hiding significant trends or variations. I advise that you plot everything you work with, no better tool than the human eye.

- As a follow up, you won't have to look at a large number of spectra (i.e. more than 2) to start spotting some data points that seem like clear outliers. They probably are, either just rogue data points, multiple objects in view (particularly a problem with IRAS) or even perfectly physical and completely unhelpful variability. If you want to fit anything you'll likely need to decide how to handle these, maybe on a case by case basis, and use a lot of care.

- If one author uses a data point from another publication, and publishes it in a machine readable form, Vizier will find both and both data points will appear here. They may be identical or slightly shifted (e.g. if they've been dereddened - a whole issue in itself). Watch out for doubling up (a tool that ~deals with this is on its way, see below).

- If you're getting a ton of warnings (especially from astroquery.IRSA) you can uncomment a few lines at the top of spectra.py. This will suppress all warnings, probably a fine thing to do but I'm leaving it on while developing the package.

- If you want there is an optional parameter windowSize in getSED which limits how large a region around the star to return SED points from. The default value is windowSize=2 (in milli-arcseconds) and is the radius within which SED points are returned. The center point is the star position according to Simbad (not always perfect, hence I suggest not making windowSize < ~1). Equally, some measurements can be quite far from the star, but still have it as the main object in the field. However, if you think you're getting contamination from a nearby source you can try decreasing it.

- There's a (blank) extra field, called 'ExtraField', in the data in which you can record any other detail you wish. If you use this a lot you might want to hardcode that parameter, or ask me to. You can write to it easily using 'rewriteExtraField(twoMassID,saveDir,entry)' where entry is whatever you want to put in there (Note that in saving it, I think this will be converted to a string. You can put whatever you want in there, including arrays, dictionaries or lists, if you can convert it to a string and back.)

- If you're just enjoying the quick access to Simbad via getInfo() you may want to use the getCoords() function. It's very simple, just takes a (Simbad recognised) name and returns the coordiantes of that object. Not used anywhere here but thought I'd leave it in.

- If you ask me what coordinate scheme everything is in my response will be "normal". Any further questions in that vein will be ignored. Don't blame me, blame yourselves astronomers.

# New features and bugs:

- Eventually I'll add some simple plotting functions.

- At the moment the only way you could keep track of all the data here is to loop through all the files. I'll have a think about the best way to keep track of all this (maybe just write a function that does that loop and records some data.

- I'll also add a tool to quickly clean data. Probably it will just limit itself to returning the most recent data point from only recent surveys, giving a much shorter, more evenly sampled, and hopefully representative set of data. There's no one way to do this so others may want to write their own/give suggestions.

- This package makes a lot of use of astroquery and the Vizier SED tool. To their authors, thank you for those invaluable resources. To everyone else, please keep in mind with whatever you're doing if it can and should be easily accesible online. Monkey work should be limited to zoologists.

- No bugs! (that I know of (yet)) A few parts of the package may go stale if not updated. If there are any new catalogs you think I should be using let me know (e.g. if I haven't updated everything to the newest Gaia data reelease).

# _ _ _ _ _ _ _ _ _ _ _
# Here be dragons!
# Below is messy code, mostly me writing and testing the package, but may add more features and explanations later.

In [ ]:
import datetime
print(str(datetime.datetime.today()).split()[0])
print(datetime.datetime.now())

In [ ]:
for key in table.meta.keys():
    print(key,': \t\t',table.meta[key])
    #print(table.meta[key])
#print(table.meta)

In [ ]:
saveDir='/Users/Zephyr/Google Drive/astro/pythonModules/SEDBuilder/spectraData'
tmName='J04174965+2829362'
table=SEDBuilder.spectra.addToMetadata(tmName,saveDir,'Hello',1,'me')
#fileTable=SEDBuilder.spectra.getSEDFromFile(tmName,saveDir)
#fileTable=astropy.io.ascii.read('/Users/Zephyr/Google Drive/astro/pythonModules/SEDBuilder/spectraData/J04174965+2829362.ecsv')
#print(fileTable)
#for key in fileTable.meta.keys():
#    print('key: ',key)
#    print(fileTable.meta[key])
#    print(type(fileTable.meta[key]))

In [ ]:
import astroquery.simbad
import astroquery.vizier
simbad=astroquery.simbad.Simbad()
simbad.add_votable_fields('sptype')
simbad.add_votable_fields('ids')
simbad.add_votable_fields('otype')
simbad.add_votable_fields('plx')
simbad.add_votable_fields('plx_bibcode')
star=simbad.query_object(v410)
print(star)
print(star.keys())
print(star['SP_TYPE'])
print(star['SP_BIBCODE'])
spTypeBib=star['SP_BIBCODE']

catalogs=astroquery.vizier.Vizier.find_catalogs(spTypeBib)
print(SEDBuilder.spectra.getRef(spTypeBib))
description=catalogs[list(catalogs.items())[0][0]].description
reference=description[description.rfind('(')+1:description.rfind(')')]
print(catalogs)
print(description)
print(reference)

In [ ]:
astroquery.simbad.Simbad.list_votable_fields()

In [ ]:
import astroquery
astroquery.simbad.Simbad.get_field_description('sp')